# Hybrid search demo

In [2]:
import os, pandas as pd
from sqlalchemy import create_engine, text

In [3]:
username = 'demo'
password = 'demo'
hostname = os.getenv('IRIS_HOSTNAME', 'localhost')
port = '1972' 
namespace = 'USER'
CONNECTION_STRING = f"iris://{username}:{password}@{hostname}:{port}/{namespace}"

In [4]:
engine = create_engine(CONNECTION_STRING)
connection = engine.connect()

In [ ]:
## Creating the table

run the `sql_demo.ipynb` notebook first

```SQL
LOAD DATA FROM FILE '~/data/scotch_review.csv'
        COLUMNS (
            id INT,
            name VARCHAR(255),
            category VARCHAR(255),
            review_point INT,
            price DOUBLE,
            currency VARCHAR(10),
            description VARCHAR(2000)
        )
        INTO hybrid.scotch_reviews (name, category, review_point, price, description)
            VALUES (name, category, review_point, price, description)
        USING { "from" : { "file" : { "header" : 1 } } }
```

TODO: fix issue with DB-API parsing and include file in container

In [5]:
res = connection.execute(text("""CREATE INDEX ifind ON scotch_reviews(description) AS %iFind.Index.Basic"""))

In [27]:
from sentence_transformers import SentenceTransformer

model = SentenceTransformer('all-MiniLM-L6-v2') 
search_vector = model.encode("vanilla", normalize_embeddings=True).tolist()

In [40]:
sql = text("""
    WITH 
    
    filtered_text AS (
       SELECT %ID AS ID, * FROM scotch_reviews
       WHERE %ID %FIND search_index(ifind, 'vanilla')
    ),

    filtered_vec AS (
        SELECT TOP 100 %ID AS ID, * FROM scotch_reviews
        ORDER BY vector_cosine(description_vector, TO_VECTOR(:search_vec ,DOUBLE)) DESC
    ),
    
    scored AS (
       SELECT name, category, description,
          hybrid.scotchreviews_ifindrank(ID, 'vanilla') AS IFindScore,
          vector_cosine(description_vector, TO_VECTOR(:search_vec ,DOUBLE)) AS VectorScore
       FROM filtered_text
    ), 
    
    with_rank AS (
       SELECT *,
          RANK() OVER (ORDER BY IFindScore DESC) AS IFindRank,
          RANK() OVER (ORDER BY VectorScore DESC) AS VectorRank
       FROM scored
    ),

    -- using k = 10
    full_score AS (
        SELECT *, (1/(IFindRank + 10) + 1/(VectorRank + 10)) AS Score
        FROM with_rank
    )
    
    SELECT TOP 10 * 
    FROM full_score 
    ORDER BY Score desc""")

result = connection.execute(sql, { "search_vec": str(search_vector) }).fetchall()
df = pd.DataFrame(result)
pd.set_option('display.max_colwidth', None)  # Easier to read description
df.head()

Exception: Incorrect number of parameters: 1/7/2